# Notebook 2: Embedding Training

## Goal
Now that we have our clean corpora, we need to turn words into vectors. We will train two separate embedding spaces:

1.  **Hieroglyphic Space**: Using **FastText**.
    -   *Why?* Hieroglyphs (and their transliterations) are morphologically rich. FastText breaks words into sub-character n-grams (e.g., "nfr" -> "<nf", "nfr", "fr>"), allowing the model to understand the structure of the words better than standard Word2Vec.
2.  **English Space**: Using **Word2Vec** (Skip-gram).
    -   *Why?* Standard Word2Vec is sufficient for English. We train it on the *same* parallel corpus to ensure the domain (Egyptology) matches perfectly.

## Steps
1.  Load the clean corpora.
2.  Train Hieroglyphic FastText model.
3.  Train English Word2Vec model.
4.  Visualize the training loss (if available) or inspect similar words to verify quality.

In [1]:
import os
import pickle
import logging
from gensim.models import FastText, Word2Vec
from gensim.utils import simple_preprocess

# Setup logging to see training progress
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Configuration
DATA_DIR = "data"
MODELS_DIR = "models"
CLEAN_CORPUS_FILE = os.path.join(DATA_DIR, "clean_corpora.pkl")
HIEROGLYPHIC_MODEL_FILE = os.path.join(MODELS_DIR, "hieroglyphic_fasttext.model")
ENGLISH_MODEL_FILE = os.path.join(MODELS_DIR, "english_word2vec.model")

# Hyperparameters
VECTOR_SIZE = 100
WINDOW = 5
MIN_COUNT = 2
EPOCHS = 50

## 1. Load Data

In [2]:
print(f"Loading corpora from {CLEAN_CORPUS_FILE}...")
with open(CLEAN_CORPUS_FILE, 'rb') as f:
    corpora = pickle.load(f)

hier_sentences = [s.split() for s in corpora['hieroglyphic']]
eng_sentences = [s.split() for s in corpora['english']]

print(f"Loaded {len(hier_sentences)} hieroglyphic sentences and {len(eng_sentences)} English sentences.")

Loading corpora from data/clean_corpora.pkl...
Loaded 12773 hieroglyphic sentences and 12773 English sentences.


## 2. Train Hieroglyphic FastText

We use the Skip-gram model (`sg=1`) because it generally produces better embeddings for smaller datasets.

In [3]:
print("Training Hieroglyphic FastText model...")
hier_model = FastText(
    sentences=hier_sentences,
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=1,  # Skip-gram
    epochs=EPOCHS,
    seed=42
)

print("Saving Hieroglyphic model...")
hier_model.save(HIEROGLYPHIC_MODEL_FILE)
print("Done.")

2025-11-19 09:25:45,315 : INFO : collecting all words and their counts


2025-11-19 09:25:45,315 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2025-11-19 09:25:45,322 : INFO : PROGRESS: at sentence #10000, processed 54462 words, keeping 6287 word types


2025-11-19 09:25:45,324 : INFO : collected 7174 word types from a corpus of 70267 raw words and 12773 sentences


2025-11-19 09:25:45,324 : INFO : Creating a fresh vocabulary


...

2025-11-19 09:25:51,684 : INFO : not storing attribute vectors


2025-11-19 09:25:51,684 : INFO : not storing attribute buckets_word


2025-11-19 09:25:51,684 : INFO : not storing attribute cum_table


2025-11-19 09:25:51,687 : INFO : saved models/hieroglyphic_fasttext.model


Saving Hieroglyphic model...
Done.


## 3. Train English Word2Vec

In [4]:
print("Training English Word2Vec model...")
eng_model = Word2Vec(
    sentences=eng_sentences,
    vector_size=VECTOR_SIZE,
    window=WINDOW,
    min_count=MIN_COUNT,
    sg=1,  # Skip-gram
    epochs=EPOCHS,
    seed=42
)

print("Saving English model...")
eng_model.save(ENGLISH_MODEL_FILE)
print("Done.")

2025-11-19 09:25:51,691 : INFO : collecting all words and their counts


2025-11-19 09:25:51,692 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2025-11-19 09:25:51,702 : INFO : PROGRESS: at sentence #10000, processed 95814 words, keeping 6847 word types


2025-11-19 09:25:51,706 : INFO : collected 7800 word types from a corpus of 123355 raw words and 12773 sentences


2025-11-19 09:25:51,706 : INFO : Creating a fresh vocabulary


...

2025-11-19 09:25:57,915 : INFO : Word2Vec lifecycle event {'params': 'Word2Vec<vocab=4177, vector_size=100, alpha=0.025>', 'datetime': '2025-11-19T09:25:57.915216', 'gensim': '4.3.3', 'python': '3.12.3 (main, Jun  1 2025, 04:19:33) [Clang 17.0.0 (clang-1700.0.13.5)]', 'platform': 'macOS-26.1-arm64-arm-64bit', 'event': 'created'}


2025-11-19 09:25:57,915 : INFO : Word2Vec lifecycle event {'fname_or_handle': 'models/english_word2vec.model', 'separately': 'None', 'sep_limit': 10485760, 'ignore': frozenset(), 'datetime': '2025-11-19T09:25:57.915543', 'gensim': '4.3.3', 'python': '3.12.3 (main, Jun  1 2025, 04:19:33) [Clang 17.0.0 (clang-1700.0.13.5)]', 'platform': 'macOS-26.1-arm64-arm-64bit', 'event': 'saving'}


2025-11-19 09:25:57,915 : INFO : not storing attribute cum_table


2025-11-19 09:25:57,918 : INFO : saved models/english_word2vec.model


Saving English model...
Done.


## 4. Sanity Check

Let's check the "most similar" words for a few concepts to see if the individual spaces learned anything meaningful.

In [5]:
def check_similarity(model, word, title):
    print(f"\n--- {title}: '{word}' ---")
    try:
        similar = model.wv.most_similar(word, topn=5)
        for w, score in similar:
            print(f"{w}: {score:.3f}")
    except KeyError:
        print(f"Word '{word}' not in vocabulary.")

# Check Hieroglyphic (e.g., 'nfr' = good/beautiful, 'ra' = sun god)
check_similarity(hier_model, 'nfr', "Hieroglyphic")
check_similarity(hier_model, 'ra', "Hieroglyphic")

# Check English
check_similarity(eng_model, 'god', "English")
check_similarity(eng_model, 'king', "English")


--- Hieroglyphic: 'nfr' ---
nfrꞽ: 0.744
nfr.n: 0.708
nfr.w: 0.649
nfr.tꞽ: 0.649
snfr: 0.645

--- Hieroglyphic: 'ra' ---
zꜣ: 0.228
ꞽm.ꞽ-rd: 0.227
ḫꜣ.w: 0.213
ꞽri̯.w: 0.205
hꜣi̯.y: 0.191

--- English: 'god' ---
chentjtkaues: 0.524
herwer: 0.515
balmier: 0.507
tjenetholydom: 0.499
jti: 0.499

--- English: 'king' ---
sheschi: 0.593
captain: 0.561
file: 0.546
heneni: 0.544
chui: 0.541
